In [1]:
import pandas as pd
import numpy as np
import json
import turbodbc

In [2]:
def create_connection_string_turbo(server, database):
    options = turbodbc.make_options(prefer_unicode=True)
    constr = 'Driver={ODBC Driver 13 for SQL Server};Server=' + \
        server + ';Database=' + database + ';Trusted_Connection=yes;'
    con = turbodbc.connect(connection_string=constr, turbodbc_options=options)
    return con

In [3]:
with open("products_detail.json") as f:
    js = json.load(f)

In [4]:
df = pd.DataFrame(js)

In [5]:
df['EAN'] = df.details.apply(lambda x: x['EAN'] if 'EAN' in x.keys() else np.nan)

In [6]:
df['Gewicht pro Stück'] = df.details.apply(lambda x: x['Gewicht pro Stück'] if 'Gewicht pro Stück' in x.keys() else np.nan)

In [7]:
df['Herstellerartikelnummer'] = df.details.apply(
    lambda x: x['Herstellerartikelnummer'] if 'Herstellerartikelnummer' in x.keys() else np.nan)

In [8]:
con = create_connection_string_turbo("CRHBUSADWH51", "Operations")

In [9]:
query = """
set nocount on;

SELECT  [Artikelnummer]
      ,[Artikel Beschreibung]
      ,[Lieferantenname]
      ,[Bruttoverkauf]
      ,[Standardkosten]
      ,[Lieferantennummer]
      ,[Barcode]
into #temp
FROM [AnalystCM].[dbo].[LOOKUP_ArtikelstammCRHT]
where 
--Artikelgruppe between 5000 and 5999 
[Einmalartikel (J/N)] = 0 and [Artikel Status] = '02'

select idItemOrigin
,AvgPrice = Avg(Sales / case when Quantity = 0 then NULL else Quantity end)
,WAvgPrice = sum(Sales) / case when sum(Quantity) = 0 then NULL else sum(Quantity) end
into #avgprice
from crhbusadwh01.infopool.fact.sales
where date > dateadd(month,-12, getdate())
group by idItemOrigin

select [Artikelnummer]
      ,[Artikel Beschreibung]
      ,[Lieferantenname]
      ,[Bruttoverkauf]
      ,[Standardkosten]
      ,EAN = Barcode
      ,[ArtikelnummerLieferant] = [Lieferantennummer]
      ,Durchschnittspreis = AvgPrice
      ,GDurchschnittspreis = WAvgPrice
from #temp t
    left join #avgprice a on t.Artikelnummer = a.idItemOrigin collate Latin1_General_CI_AS

drop table #temp
drop table #avgprice
"""

In [10]:
df_server = pd.read_sql(query, con)

In [11]:
df = df.merge(df_server, how='left', left_on="ean", right_on='EAN')

In [12]:
matched_df = df[~pd.isna(df['Artikelnummer'])]

In [13]:
matched_df.columns

Index(['cat1', 'cat2', 'cat3', 'details', 'ean', 'header', 'id', 'price',
       'EAN_x', 'Gewicht pro Stück', 'Herstellerartikelnummer',
       'Artikelnummer', 'Artikel Beschreibung', 'Lieferantenname',
       'Bruttoverkauf', 'Standardkosten', 'EAN_y', 'ArtikelnummerLieferant',
       'Durchschnittspreis', 'GDurchschnittspreis'],
      dtype='object')

In [14]:
matched_df[['cat1', 'cat2', 'cat3', 'ean', 'header', 'id', 'price',
            'Herstellerartikelnummer','Artikelnummer',
            'Artikel Beschreibung', 'Lieferantenname',
            'Bruttoverkauf', 'Standardkosten', 'ArtikelnummerLieferant',
            'Durchschnittspreis', 'GDurchschnittspreis']].to_clipboard(index=False)